In [7]:
import json
import xmltodict
import pandas as pd
import numpy as np

In [8]:
# Fonte
# https://www.al.sp.gov.br/dados-abertos/grupo/2;jsessionid=4D6AFD92F42D3230C23712D324C08157

In [9]:
# Converte XML para JSON

In [10]:
with open("dados/despesas_gabinetes_8_junho.xml", 'r') as f:
    xmlString = f.read()

In [11]:
jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)

In [12]:
with open("output.json", 'w') as f:
    f.write(jsonString)

In [13]:
arquivo_json = open('output.json', 'r')

In [14]:
dados_json = json.load(arquivo_json)

In [8]:
dados_json

{'despesas': {'despesa': [{'Ano': '2015',
    'Matricula': '300257',
    'Mes': '3',
    'Valor': '200.0',
    'CNPJ': '71806251000106',
    'Deputado': 'ABELARDO CAMARINHA',
    'Tipo': 'A - COMBUSTÍVEIS E LUBRIFICANTES',
    'Fornecedor': 'AUTO POSTO DE SERVIÇOS MARV LTDA'},
   {'Ano': '2015',
    'Matricula': '300257',
    'Mes': '3',
    'Valor': '295.4',
    'CNPJ': '68064740000125',
    'Deputado': 'ABELARDO CAMARINHA',
    'Tipo': 'E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO',
    'Fornecedor': 'PAPER FACE COMERCIAL LTDA'},
   {'Ano': '2015',
    'Matricula': '300257',
    'Mes': '3',
    'Valor': '2850.0',
    'CNPJ': '22145388877',
    'Deputado': 'ABELARDO CAMARINHA',
    'Tipo': 'N - MORADIA',
    'Fornecedor': 'LARA SERVINO VARGAS ABREU'},
   {'Ano': '2015',
    'Matricula': '300257',
    'Mes': '4',
    'Valor': '100.0',
    'CNPJ': '10915883000113',
    'Deputado': 'ABELARDO CAMARINHA',
    'Tipo': 'A - COMBUSTÍVEIS E LUBRIFICANTES',
    'Fornecedor': 'CENTR

In [8]:
dados_json['despesas']['despesa']

[{'Ano': '2015',
  'CNPJ': '71806251000106',
  'Deputado': 'ABELARDO CAMARINHA',
  'Fornecedor': 'AUTO POSTO DE SERVIÇOS MARV LTDA',
  'Matricula': '300257',
  'Mes': '3',
  'Tipo': 'A - COMBUSTÍVEIS E LUBRIFICANTES',
  'Valor': '200.0'},
 {'Ano': '2015',
  'CNPJ': '68064740000125',
  'Deputado': 'ABELARDO CAMARINHA',
  'Fornecedor': 'PAPER FACE COMERCIAL LTDA',
  'Matricula': '300257',
  'Mes': '3',
  'Tipo': 'E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO',
  'Valor': '295.4'},
 {'Ano': '2015',
  'CNPJ': '22145388877',
  'Deputado': 'ABELARDO CAMARINHA',
  'Fornecedor': 'LARA SERVINO VARGAS ABREU',
  'Matricula': '300257',
  'Mes': '3',
  'Tipo': 'N - MORADIA',
  'Valor': '2850.0'},
 {'Ano': '2015',
  'CNPJ': '10915883000113',
  'Deputado': 'ABELARDO CAMARINHA',
  'Fornecedor': 'CENTRO AUTOMOTIVO TOP 23 LTDA',
  'Matricula': '300257',
  'Mes': '4',
  'Tipo': 'A - COMBUSTÍVEIS E LUBRIFICANTES',
  'Valor': '100.0'},
 {'Ano': '2015',
  'CNPJ': '71806251000106',
  'Deputado': 

In [15]:
lista = dados_json['despesas']['despesa']

In [16]:
df_despesas = pd.DataFrame(lista, columns = ['Ano', 'CNPJ', 'Deputado', 'Fornecedor', 'Matricula', 'Mes', 'Tipo', 'Valor'])

In [17]:
df_despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453000 entries, 0 to 452999
Data columns (total 8 columns):
Ano           453000 non-null object
CNPJ          443353 non-null object
Deputado      453000 non-null object
Fornecedor    453000 non-null object
Matricula     453000 non-null object
Mes           453000 non-null object
Tipo          453000 non-null object
Valor         453000 non-null object
dtypes: object(8)
memory usage: 27.6+ MB


In [18]:
df_despesas['Valor'] = df_despesas.Valor.astype('float64')

In [19]:
df_despesas['CNPJ'] = df_despesas.CNPJ.astype('str')

In [20]:
def limpeza(linha):
    documento = linha["CNPJ"]
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf_"

In [21]:
df_despesas['cnpj_raiz_fornecedor'] = df_despesas.apply(limpeza, axis = 1)

In [16]:
df_despesas.reset_index()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
0,0,2015,71806251000106,ABELARDO CAMARINHA,AUTO POSTO DE SERVIÇOS MARV LTDA,300257,3,A - COMBUSTÍVEIS E LUBRIFICANTES,200.00,71806251
1,1,2015,68064740000125,ABELARDO CAMARINHA,PAPER FACE COMERCIAL LTDA,300257,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,295.40,68064740
2,2,2015,22145388877,ABELARDO CAMARINHA,LARA SERVINO VARGAS ABREU,300257,3,N - MORADIA,2850.00,documento_cpf_
3,3,2015,10915883000113,ABELARDO CAMARINHA,CENTRO AUTOMOTIVO TOP 23 LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,100.00,10915883
4,4,2015,71806251000106,ABELARDO CAMARINHA,AUTO POSTO DE SERVIÇOS MARV LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,195.01,71806251
5,5,2015,06222505000187,ABELARDO CAMARINHA,WAGNER HUBERT EPP,300257,4,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,200.00,06222505
6,6,2015,10455524000120,ABELARDO CAMARINHA,ROGÉRIO GODOY CARIMBOS - ME,300257,4,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,60.00,10455524
7,7,2015,04206050000180,ABELARDO CAMARINHA,TIM CELULAR SA,300257,4,"H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERG...",719.02,04206050
8,8,2015,14691478000110,ABELARDO CAMARINHA,AGENCIA DE SERV POSTAIS AVARÉ LTDA,300257,4,J - SERVIÇOS DE COMUNICAÇÃO,719.94,14691478
9,9,2015,22145388877,ABELARDO CAMARINHA,LARA SERVINO VARGAS ABREU,300257,4,N - MORADIA,2850.00,documento_cpf_


In [ ]:
# Soma histórico de despesas

In [17]:
df_despesas['Valor'].sum()

309869138.75999993

In [ ]:
# Soma por deputados 

In [18]:
df_despesas.groupby("Deputado").Valor.sum().reset_index().sort_values("Valor", ascending=False)

,Deputado,Valor
41,CAMPOS MACHADO,4031886.86
10,ALDO DEMARCHI,3777443.31
182,MARIA LÚCIA AMARY,3735760.69
5,AFONSO LOBATO,3707932.08
72,CÉLIA LEÃO,3659272.02
233,ROBERTO ENGLER,3649787.50
159,LUIZ CARLOS GONDIM,3643216.45
100,ENIO TATTO,3597812.46
20,ANALICE FERNANDES,3568734.57
17,ANA DO CARMO,3395167.07


In [ ]:
# Soma por fornecedor

In [42]:
df_despesas.groupby("Fornecedor").Valor.sum().reset_index().sort_values("Valor", ascending=False)

,Fornecedor,Valor
65418,TELEFONICA,2870649.22
65425,TELEFONICA BRASIL S.A.,2446829.08
46823,NEXTEL TELECOMUNICAÇÕES LTDA,2176978.83
65438,TELEFÔNICA BRASIL S.A.,2132265.41
65423,TELEFONICA BRASIL S.A,1534093.31
50429,PEDÁGIO,1289144.33
68288,VIVO S.A.,1255101.17
46824,NEXTEL TELECOMUNICAÇÕES LTDA.,1252432.18
65426,TELEFONICA BRASIL S/A,1184894.18
68279,VIVO,1071501.84


In [ ]:
# Tabela dinâmica por ano, deputado e cnpj raiz de fornecedor

In [17]:
pd.pivot_table(df_despesas,index=["Ano","Deputado", "cnpj_raiz_fornecedor"], values=["Valor"], aggfunc=np.sum)

Valor
Ano  Deputado         cnpj_raiz_fornecedor          
2002 AFANÁSIO JAZADJI 00011835               3818.22
                      00016968               1904.66
                      00024610                 93.03
                      00480676                 27.00
                      00548286                 32.50
                      00986749                  8.00
                      00994307                 39.20
                      01103297                 50.00
                      01163350                 12.00
                      01215739                  8.00
                      01220109                760.00
                      01340107                 15.00
                      01354636                718.76
                      01369052                 30.00
                      01432304                 23.95
                      01521958                  6.00
                      01545828                482.85
                      01648812                 30.01
                      01692471                 50.00
                      01992797                 49.03
                      02110034                 50.00
                      02290698                 50.00
                      02319126               2792.40
                      02430531                  4.00
                      02552071                101.80
                      02558157              11286.17
                      02630038               5800.00
                      02669912                 49.80
                      02680600                100.00
                      02725282               1303.50
...                                              ...
2018 WELSON GASPARINI 15181028                230.48
                      15492364                370.00
                      16963311                 29.15
                      18530515                300.00
                      20778191               1645.38
                      21458327                 42.88
                      26279731                738.14
                      33050196               1064.63
                      40432544                224.85
                      42591651                 35.90
                      43283811                291.60
                      45236791                477.18
                      47970033                174.72
                      48017909                529.18
                      48344097                100.00
                      49375694                200.01
                      52742491               5517.00
                      52791142                 80.00
                      56007859                 89.21
                      56022858                 59.58
                      58826041                250.56
                      59434449                734.00
                      61084018                419.99
                      61216917                161.26
                      64505431                395.02
                      64793771                128.00
                      66604620                565.01
                      71322150                116.42
                      71921514                252.20
                      documento_cpf_        14006.02

[161125 rows x 1 columns]

In [22]:
df_despesas.to_csv('resultados/despesas_dep_est_sp_8_junho.csv', index=False, decimal=',')

# Carrega arquivo salvo

In [23]:
df_despesas = pd.read_csv("resultados/despesas_dep_est_sp_8_junho.csv", sep=',',encoding = 'utf-8', converters={'CNPJ': lambda x: str(x), 'cnpj_raiz_fornecedor': lambda x: str(x)}, decimal=',')

In [ ]:
# Procura por nome de fornecedor

In [24]:
mask = df_despesas['Fornecedor'].str.contains("ITORORO", na=False)
procura = df_despesas[mask]

In [23]:
procura

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
41,2015,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,9,A - COMBUSTÍVEIS E LUBRIFICANTES,56.28,61488904
42,2015,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,9,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,3243.72,61488904
65,2016,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,3,A - COMBUSTÍVEIS E LUBRIFICANTES,210.00,61488904
66,2016,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,3,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,3148.71,61488904
105,2016,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,8,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,2454.56,61488904
119,2016,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,10,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,1374.00,61488904
134,2016,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,12,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,634.00,61488904
158,2017,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,261.00,61488904
159,2017,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,4,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,1439.00,61488904
166,2017,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,5,A - COMBUSTÍVEIS E LUBRIFICANTES,216.00,61488904


In [25]:
# Procura por CNPJ raiz

In [22]:
nome = df_despesas[(df_despesas['cnpj_raiz_fornecedor'] == '19876022')]

In [23]:
nome

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
2616,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,10,K - LOCAÇÃO DE BENS MÓVEIS,3800.0,19876022
2619,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,10,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",3500.0,19876022
2627,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,11,K - LOCAÇÃO DE BENS MÓVEIS,3800.0,19876022
2630,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,11,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",3500.0,19876022
2639,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,12,K - LOCAÇÃO DE BENS MÓVEIS,3800.0,19876022
2642,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,12,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",3500.0,19876022
2651,2016,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,1,K - LOCAÇÃO DE BENS MÓVEIS,3800.0,19876022
2654,2016,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,1,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",3500.0,19876022
2664,2016,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,2,K - LOCAÇÃO DE BENS MÓVEIS,3800.0,19876022
2667,2016,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,2,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",3500.0,19876022


In [ ]:
# CNPJ neste caso é um CPF de deputado

In [4]:
nome = df_despesas[(df_despesas['CNPJ'] == '02674980800')]
nome

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
366447,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_


In [26]:
# Separa lote para checar em https://www.al.sp.gov.br/deputados/prestacao-de-contas/

In [33]:
despesa_selecionada = df_despesas[(df_despesas['Deputado'] == 'ÍTALO CARDOSO') & 
                              (df_despesas['Ano'] == 2007) & 
                              (df_despesas['Mes'] == 3) & 
                              (df_despesas['Tipo'] == 'E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO')]

In [34]:
despesa_selecionada.Valor.sum()

2632.4100000000003

In [35]:
despesa_selecionada

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
452986,2007,00000998000124,ÍTALO CARDOSO,HIPE COMERCIO DE PAPEIS LTDA.,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,2022.30,00000998
452987,2007,00986363000148,ÍTALO CARDOSO,COME'X COMERCIO E REPRESENTAÇÃO LTDA.,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,500.00,00986363
452988,2007,03476811002103,ÍTALO CARDOSO,DIA BRASIL SOCIEDADE LTDA,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,14.85,03476811
452989,2007,43283811003841,ÍTALO CARDOSO,KALUNGA COM. E IND.GRÁFICA LTDA.,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,24.30,43283811
452990,2007,47508411090988,ÍTALO CARDOSO,CIA. BRASILEIRA DE DISTRIBUIÇÃO,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,46.19,47508411
452991,2007,61072898000106,ÍTALO CARDOSO,EMPRESA DE AGUAS PETROPOLIS PAULISTA LTDA.,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,24.77,61072898


# Seleciona despesas da atual legislatura

In [3]:
despesas_atuais = df_despesas[(df_despesas['Ano'] == 2015) | 
                              (df_despesas['Ano'] == 2016) | 
                              (df_despesas['Ano'] == 2017) | 
                              (df_despesas['Ano'] == 2018)]

In [37]:
# Quantidade geral de despesas

In [38]:
despesas_atuais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80667 entries, 0 to 450256
Data columns (total 9 columns):
Ano                     80667 non-null int64
CNPJ                    80667 non-null object
Deputado                80667 non-null object
Fornecedor              80667 non-null object
Matricula               80667 non-null int64
Mes                     80667 non-null int64
Tipo                    80667 non-null object
Valor                   80667 non-null float64
cnpj_raiz_fornecedor    80667 non-null object
dtypes: float64(1), int64(3), object(5)
memory usage: 6.2+ MB


In [39]:
# Soma das despesas

In [40]:
despesas_atuais['Valor'].sum()

75154853.75000001

In [41]:
# Média

In [42]:
despesas_atuais['Valor'].mean()

931.667890835162

In [43]:
# Mediana

In [44]:
despesas_atuais['Valor'].median()

210.0

In [45]:
# Maior despesa

In [46]:
despesas_atuais[despesas_atuais['Valor']== despesas_atuais['Valor'].max()]

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
417503,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,8,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_
417529,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,9,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_
417555,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,10,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_
417576,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,11,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_
417599,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,12,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_
417622,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,1,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_
417648,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,2,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_
417678,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,3,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_
417719,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,4,L - LOCAÇÃO DE BENS IMÓVEIS,9334.61,documento_cpf_


In [47]:
# Média por ano 

In [48]:
despesas_atuais.groupby('Ano').mean()['Valor']

Ano
2015    900.026015
2016    946.345794
2017    942.698156
2018    943.934130
Name: Valor, dtype: float64

In [49]:
# Quantidade de tipos de despesas

In [50]:
despesas_atuais['Tipo'].nunique()

13

In [51]:
# Quantidade de despesas por Tipo

In [52]:
despesas_atuais['Tipo'].value_counts()

A - COMBUSTÍVEIS E LUBRIFICANTES                                      21047
I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCOMOÇÃO                   12048
H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERGIA, ÁGUA, GÁS ETC)      10091
E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO              8497
M - MANUTENÇÃO DE BENS MÓVEIS,  IMÓVEIS, CONDOMÍNIOS E OUTROS          6333
G - ASSINATURAS DE PERIÓDICOS, PUBLICAÇÕES E INTERNET                  4802
L - LOCAÇÃO DE BENS IMÓVEIS                                            4158
K - LOCAÇÃO DE BENS MÓVEIS                                             3567
C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE VEÍCULOS ; PEDÁGIOS     3084
D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS     2836
J - SERVIÇOS DE COMUNICAÇÃO                                            2095
F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC)       1398
N - MORADIA                                                             711
Name: Tipo, 

In [53]:
# Gastos por tipo de despesa

In [54]:
despesas_atuais.groupby("Tipo").Valor.sum().reset_index().sort_values("Valor", ascending=False)

,Tipo,Valor
2,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",15524858.83
10,L - LOCAÇÃO DE BENS IMÓVEIS,14335963.01
0,A - COMBUSTÍVEIS E LUBRIFICANTES,6804230.63
6,"H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERG...",6510284.26
11,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",5624465.23
4,F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTOR...,5414173.82
9,K - LOCAÇÃO DE BENS MÓVEIS,4731538.02
3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,4637839.88
8,J - SERVIÇOS DE COMUNICAÇÃO,3796261.31
1,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,2804414.54


In [55]:
# Mais gastos em quatidade em tipo D por deputado

In [56]:
despesas_atuais[despesas_atuais['Tipo']=='D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS']['Deputado'].value_counts().head(10)

CARLÃO PIGNATARI         65
WELSON GASPARINI         62
ROBERTO ENGLER           58
GIL LANCASTER            54
MILTON VIEIRA            53
ALENCAR SANTANA BRAGA    53
EDSON GIRIBONI           48
ROBERTO MORAIS           46
ANALICE FERNANDES        45
SEBASTIÃO SANTOS         44
Name: Deputado, dtype: int64

In [57]:
# Maiores gastos em valor em tipo D por deputado

In [58]:
graficas = despesas_atuais[(despesas_atuais['Tipo'] == 'D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS')]

In [59]:
graficas.groupby("Deputado").Valor.sum().reset_index().sort_values("Valor", ascending=False).head(10)

,Deputado,Valor
10,ANA DO CARMO,319000.00
121,SEBASTIÃO SANTOS,315203.66
111,ROBERTO ENGLER,311533.00
19,BETH SAHÃO,311350.00
75,LECI BRANDÃO,308715.00
95,MILTON VIEIRA,304400.75
116,ROGÉRIO NOGUEIRA,304000.00
106,RAMALHO DA CONSTRUÇÃO,301030.00
48,ENIO TATTO,295420.00
112,ROBERTO MASSAFERA,295000.00


In [60]:
# Graficas usadas por ANA do CARMO

In [61]:
ana_grafica = graficas[(graficas['Deputado'] == 'ANA DO CARMO')]

In [62]:
ana_grafica.reset_index()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
0,22134,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,1,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
1,22137,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,2,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
2,22142,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,3,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
3,22145,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,4,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
4,22151,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,5,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
5,22155,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,6,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
6,22161,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,7,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
7,22166,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,8,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
8,22170,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,9,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465
9,22175,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,10,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",8000.0,08026465


In [50]:
'''
AGB PACK COMERCIO DE EMBALAGENS E SERVICOS
VIVIANE DE CASSIA FERREIRA 
R VISCONDE DE PARNAIBA S/N - SAO PAULO - SP

ANNA GIULIA CARMO BUSCATTI
EPP
R MADRE DE DEUS 1285 APT 101 - SAO PAULO - SP
'''

'\nAGB PACK COMERCIO DE EMBALAGENS E SERVICOS\nVIVIANE DE CASSIA FERREIRA \nR VISCONDE DE PARNAIBA S/N - SAO PAULO - SP\n\nANNA GIULIA CARMO BUSCATTI\nEPP\nR MADRE DE DEUS 1285 APT 101 - SAO PAULO - SP\n'

In [ ]:
# Mais gastos em quatidade em tipo L por deputado

In [63]:
despesas_atuais[despesas_atuais['Tipo']=='L - LOCAÇÃO DE BENS IMÓVEIS']['Deputado'].value_counts().head(10)

CAMPOS MACHADO           176
ROBERTO MASSAFERA        120
FERNANDO CURY            113
WELLINGTON MOURA         103
MARTA COSTA               95
ALENCAR SANTANA BRAGA     93
BARROS MUNHOZ             90
ENIO TATTO                88
GILENO GOMES              84
SEBASTIÃO SANTOS          80
Name: Deputado, dtype: int64

In [64]:
# Maiores gastos em valor em tipo L por deputado

In [65]:
locacao_imoveis = despesas_atuais[(despesas_atuais['Tipo'] == 'L - LOCAÇÃO DE BENS IMÓVEIS')] 

In [66]:
locacao_imoveis.groupby("Deputado").Valor.sum().reset_index().sort_values("Valor", ascending=False).head(10)

,Deputado,Valor
18,CAMPOS MACHADO,663104.44
24,CELINO CARDOSO,419093.87
106,WELLINGTON MOURA,379398.50
3,ALDO DEMARCHI,341921.75
7,ANA DO CARMO,327239.24
43,FERNANDO CAPEZ,316297.42
4,ALENCAR SANTANA BRAGA,304066.44
44,FERNANDO CURY,300000.00
10,ANDRÉ SOARES,293443.59
34,DAVI ZAIA,279133.17


In [67]:
# Imoveis usados por CAMPOS MACHADO

In [68]:
imoveis_campos = locacao_imoveis[(locacao_imoveis['Deputado'] == 'CAMPOS MACHADO')]

In [69]:
imoveis_campos.reset_index()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
0,74203,2015,07813996000101,CAMPOS MACHADO,CMV AUTOMAÇÃO S/A,300217,1,L - LOCAÇÃO DE BENS IMÓVEIS,6800.0,07813996
1,74204,2015,13033477844,CAMPOS MACHADO,CLAUDIA SALLUM YAZBEK,300217,1,L - LOCAÇÃO DE BENS IMÓVEIS,1400.0,documento_cpf_
2,74205,2015,15195476845,CAMPOS MACHADO,IVETE ABBUD DIB,300217,1,L - LOCAÇÃO DE BENS IMÓVEIS,3400.0,documento_cpf_
3,74206,2015,15306494803,CAMPOS MACHADO,CLARICE ABBUD GABRIEL,300217,1,L - LOCAÇÃO DE BENS IMÓVEIS,3400.0,documento_cpf_
4,74214,2015,07813996000101,CAMPOS MACHADO,CMV AUTOMAÇÃO S/A,300217,2,L - LOCAÇÃO DE BENS IMÓVEIS,6800.0,07813996
5,74215,2015,13033477844,CAMPOS MACHADO,CLAUDIA SALLUM YAZBEK,300217,2,L - LOCAÇÃO DE BENS IMÓVEIS,1400.0,documento_cpf_
6,74216,2015,15195476845,CAMPOS MACHADO,IVETE ABBUD DIB,300217,2,L - LOCAÇÃO DE BENS IMÓVEIS,3400.0,documento_cpf_
7,74217,2015,15306494803,CAMPOS MACHADO,CLARICE ABBUD GABRIEL,300217,2,L - LOCAÇÃO DE BENS IMÓVEIS,3400.0,documento_cpf_
8,74223,2015,07813996000101,CAMPOS MACHADO,CMV AUTOMAÇÃO S/A,300217,3,L - LOCAÇÃO DE BENS IMÓVEIS,6800.0,07813996
9,74224,2015,13033477844,CAMPOS MACHADO,CLAUDIA SALLUM YAZBEK,300217,3,L - LOCAÇÃO DE BENS IMÓVEIS,1400.0,documento_cpf_


In [57]:
'''
+ CMV AUTOMACAO S/A.
R CONSELHEIRO ZACARIAS, 201
VICTOR NACIM ABBUD 
GUILNAR ATALLAH ABBUD

+ CLAUDIA SALLUM YAZBEK

+ IVETE ABBUD DIB

+ CLARICE ABBUD GABRIEL

+ NORMA ABBUD SALLUM
'''

'\n+ CMV AUTOMACAO S/A.\nR CONSELHEIRO ZACARIAS, 201\nVICTOR NACIM ABBUD \nGUILNAR ATALLAH ABBUD\n\n+ CLAUDIA SALLUM YAZBEK\n\n+ IVETE ABBUD DIB\n\n+ CLARICE ABBUD GABRIEL\n\n+ NORMA ABBUD SALLUM\n'

In [3]:
# Pega CPF de deputados e cruza com despesas - para saber se há fornecedores com CPF de deputados

In [24]:
deputadossp_2014 = pd.read_csv("resultados/deputados_eleitos_2014_sp.csv",sep=',',encoding = 'utf-8', converters={'cpf': lambda x: str(x)})

In [25]:
despesas_deputados_cpf = pd.merge(deputadossp_2014, despesas_atuais, left_on='cpf', right_on='CNPJ')

In [26]:
despesas_deputados_cpf.reset_index()

,index,uf,cargo,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
0,0,SP,DEPUTADO ESTADUAL,IGOR SOARES EBERT,250000000208,28223302802,IGOR SOARES,PTN,ELEITO POR QP,2015,28223302802,IGOR SOARES,MARIA ELISA INTERNODATO NUNES VIEIRA,300554,4,L - LOCAÇÃO DE BENS IMÓVEIS,5800.00,documento_cpf_
1,1,SP,DEPUTADO ESTADUAL,IGOR SOARES EBERT,250000000208,28223302802,IGOR SOARES,PTN,ELEITO POR QP,2015,28223302802,IGOR SOARES,MARIA ELISA INTERNODATO NUNES VIEIRA,300554,7,L - LOCAÇÃO DE BENS IMÓVEIS,5800.00,documento_cpf_
2,2,SP,DEPUTADO ESTADUAL,IGOR SOARES EBERT,250000000208,28223302802,IGOR SOARES,PTN,ELEITO POR QP,2015,28223302802,IGOR SOARES,MARIA ELISA INTERNODATO NUNES VIEIRA,300554,7,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",46.45,documento_cpf_
3,3,SP,DEPUTADO ESTADUAL,IGOR SOARES EBERT,250000000208,28223302802,IGOR SOARES,PTN,ELEITO POR QP,2016,28223302802,IGOR SOARES,MARIA ELISA INTERNODATO NUNES VIEIRA,300554,1,L - LOCAÇÃO DE BENS IMÓVEIS,5800.00,documento_cpf_
4,4,SP,DEPUTADO ESTADUAL,IGOR SOARES EBERT,250000000208,28223302802,IGOR SOARES,PTN,ELEITO POR QP,2016,28223302802,IGOR SOARES,MARIA ELISA INTERNODATO NUNES VIEIRA,300554,1,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",46.45,documento_cpf_
5,5,SP,DEPUTADO ESTADUAL,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.00,documento_cpf_
